In [2]:
import pandas as pd

In [53]:
df = pd.read_excel('cashback.xlsx')
df = df[df['возраст'] >= 18]
#23
expenses_cats = list(df.columns.values)[6:21]
cats = [c.replace('оборот_', '') for c in expenses_cats]

df[expenses_cats] = df[expenses_cats].fillna(0)
df[expenses_cats]

,оборот_аптеки,оборот_рестораны,оборот_одежда_и_обувь,оборот_автоуслуги,оборот_супермаркеты,оборот_такси,оборот_красота,оборот_развлечения,оборот_жд_билеты,оборот_образование,оборот_дом_и_ремонт,оборот_спорттовары,оборот_животные,оборот_цветы,оборот_фастфуд
0,18993.0,99900.0,3294.0,0.0,397450.0,86037.0,96570.0,27420.0,0.0,0.0,9112.0,2100.0,0.0,660.0,27919.0
1,37807.0,92424.0,0.0,6750.0,219394.0,109221.0,80520.0,76950.0,0.0,0.0,63936.0,5550.0,37147.0,0.0,87510.0
2,20791.0,124800.0,51246.0,67490.0,479201.0,100395.0,66165.0,113100.0,0.0,0.0,16413.0,4215.0,8122.0,0.0,7668.0
3,131129.0,18060.0,78120.0,2400.0,532117.0,63909.0,59280.0,0.0,0.0,0.0,240492.0,0.0,18297.0,0.0,14520.0
4,44096.0,198480.0,19080.0,147705.0,286070.0,69279.0,178920.0,0.0,0.0,0.0,58884.0,13200.0,0.0,0.0,88527.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59821,657.0,54375.0,0.0,0.0,42113.0,46719.0,0.0,6000.0,0.0,0.0,0.0,0.0,0.0,0.0,24894.0
59822,0.0,128070.0,0.0,0.0,37164.0,86751.0,1800.0,5400.0,0.0,0.0,0.0,0.0,0.0,0.0,14190.0
59823,5919.0,26085.0,0.0,0.0,32419.0,31272.0,0.0,27300.0,2700.0,0.0,2997.0,0.0,0.0,0.0,39984.0
59824,1705.0,111690.0,0.0,0.0,16539.0,47529.0,1950.0,2778.0,0.0,0.0,0.0,0.0,0.0,0.0,23385.0


In [55]:
countries = df.groupby('ключ_клиента')['регион_проживания'].first()
columns = {'регион': countries}

for c in cats:
    with_cashback = df[df['активация_кэшбэка_'+c] == 1].groupby('ключ_клиента')['оборот_'+c].mean()[df.groupby('ключ_клиента')['оборот_'+c].sum() != 0]
    print(c+':', df[df['активация_кэшбэка_'+c] == 1].groupby('ключ_клиента')['оборот_'+c].mean().count() - with_cashback.count())
    without_cashback = df[df['активация_кэшбэка_'+c] != 1].groupby('ключ_клиента')['оборот_'+c].mean()[df.groupby('ключ_клиента')['оборот_'+c].sum() != 0]
    print(c+':', df[df['активация_кэшбэка_'+c] != 1].groupby('ключ_клиента')['оборот_'+c].mean().count() - without_cashback.count())
    summ = (with_cashback + without_cashback) / 100
    columns['оборот_'+c+'_с_активацией'] = with_cashback / summ
    columns['оборот_'+c+'_без_активации'] = without_cashback / summ
    columns['повышение_оборота_'+c] = 0

active = pd.DataFrame(columns).fillna(0).groupby('регион').mean()

for c in cats:
    summ = (active['оборот_'+c+'_с_активацией'] + active['оборот_'+c+'_без_активации']) / 100
    active['оборот_'+c+'_с_активацией'] = (active['оборот_'+c+'_с_активацией'] / summ).fillna(0)
    active['оборот_'+c+'_без_активации'] = (active['оборот_'+c+'_без_активации'] / summ).fillna(0)
    active['повышение_оборота_'+c] = active.apply(lambda l:  1 if l['оборот_'+c+'_с_активацией'] - l['оборот_'+c+'_без_активации'] > 5 else 0, axis=1)

active.to_excel('ds12.xlsx')
active

аптеки: 445
аптеки: 805
рестораны: 687
рестораны: 802
одежда_и_обувь: 824
одежда_и_обувь: 959
автоуслуги: 187
автоуслуги: 4537
супермаркеты: 463
супермаркеты: 469
такси: 518
такси: 2146
красота: 950
красота: 3202
развлечения: 628
развлечения: 1605
жд_билеты: 1084
жд_билеты: 6006
образование: 344
образование: 5707
дом_и_ремонт: 194
дом_и_ремонт: 1227
спорттовары: 797
спорттовары: 4770
животные: 120
животные: 5827
цветы: 324
цветы: 3440
фастфуд: 19
фастфуд: 587


,оборот_аптеки_с_активацией,оборот_аптеки_без_активации,повышение_оборота_аптеки,оборот_рестораны_с_активацией,оборот_рестораны_без_активации,повышение_оборота_рестораны,оборот_одежда_и_обувь_с_активацией,оборот_одежда_и_обувь_без_активации,повышение_оборота_одежда_и_обувь,оборот_автоуслуги_с_активацией,...,повышение_оборота_спорттовары,оборот_животные_с_активацией,оборот_животные_без_активации,повышение_оборота_животные,оборот_цветы_с_активацией,оборот_цветы_без_активации,повышение_оборота_цветы,оборот_фастфуд_с_активацией,оборот_фастфуд_без_активации,повышение_оборота_фастфуд
регион,,,,,,,,,,,,,,,,,,,,,
Алтайский край,58.882807,41.117193,1,47.782693,52.217307,0,67.270550,32.729450,1,62.791240,...,0,47.142975,52.857025,0,45.707027,54.292973,0,73.654404,26.345596,1
Амурская область,56.341063,43.658937,1,48.305970,51.694030,0,46.527656,53.472344,0,59.549499,...,0,41.538757,58.461243,0,41.906406,58.093594,0,0.000000,0.000000,0
Архангельская область,63.182647,36.817353,1,51.322223,48.677777,0,62.363377,37.636623,1,52.588647,...,0,43.540857,56.459143,0,77.475721,22.524279,1,52.724134,47.275866,1
Астраханская область,61.562033,38.437967,1,52.613264,47.386736,1,49.639363,50.360637,0,54.519853,...,1,25.173611,74.826389,0,23.106018,76.893982,0,26.651201,73.348799,0
Белгородская область,59.963924,40.036076,1,48.590230,51.409770,0,50.070286,49.929714,0,63.782692,...,0,54.055963,45.944037,1,39.198444,60.801556,0,0.000000,0.000000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Хабаровский край,55.904837,44.095163,1,55.812706,44.187294,1,53.673661,46.326339,1,56.357675,...,0,52.676282,47.323718,1,45.546359,54.453641,0,78.888681,21.111319,1
Ханты-Мансийский автономный округ,58.133453,41.866547,1,52.354673,47.645327,0,54.766806,45.233194,1,57.364810,...,0,36.942117,63.057883,0,48.117591,51.882409,0,52.299388,47.700612,0
Челябинская область,57.304904,42.695096,1,55.565617,44.434383,1,51.665348,48.334652,0,61.436908,...,0,44.198760,55.801240,0,49.204642,50.795358,0,35.843754,64.156246,0


In [113]:
import plotly.express as px
import plotly.graph_objects as go

#46
dat = pd.read_excel('ds12.xlsx')
ccs = []
#1-46
for i in range(1, 46, 3):
    ccs.append(dat.columns[i])
    ccs.append(dat.columns[i+1])
for c in cats:
    dat = dat.drop('повышение_оборота_'+c, axis=1)
msc = dat[dat['регион'] == 'Москва'].drop('регион', axis=1).transpose()
regs =dat[dat['регион'] != 'Москва'].drop('регион', axis=1).mean().to_frame()
res = pd.concat([msc, regs])

#px.bar(y=msc.iloc[0], x=ccs)
#px.bar(y=regs.iloc[0], x=ccs)
#px.bar(res, barmode='group', labels={'variable': ['Москва', 'Регионы']})

fig = go.Figure()
fig.add_trace(go.Bar(y=dat.iloc[0], x=ccs, name='Москва'))
fig.add_trace(go.Bar(y=dat.iloc[1], x=ccs, name='Регионы'))
fig.show()

In [26]:
import plotly.express as px

data = ['14', '11', '10', '9', '8', '8', '8', '7', '6', '6', '5', '5', '5', '5', '4', '4', '4', '4', '4', '4', '4', '3', '3', '3', '3', '3', '3', '3', '3', '2', '2', '2', '1', '1', '1', '1']; data.reverse()
data = [int(i) for i in data]

X = list(range(len(data)))

px.line(y=data, x=X)


In [47]:
#counts = df['регион_проживания'].value_counts() / 6
#import plotly.express as px

#Y = [counts['Москва'], counts['Санкт-Петербург'], counts[[c for c in counts.keys() if c not in ['Москва', 'Санкт-Петербург']]].sum()]
#X = ['Москва', 'Санкт-Петербург', 'Регионы']
#counts = pd.DataFrame({'Регион': X, 'Население': Y})

#px.pie(counts, values='Население', names='Регион')

counts = df['регион_проживания'].value_counts() / 6
import plotly.express as px
import plotly.graph_objects as go

Y = [counts['Москва'], counts['Санкт-Петербург'], counts[[c for c in counts.keys() if c not in ['Москва', 'Санкт-Петербург']]].sum()]
X = ['Москва', 'Санкт-Петербург', 'Регионы']
colors = ['gold', 'lightgreen', 'mediumturquoise']
counts = pd.DataFrame({'Регион': X, 'Население': Y, 'colors': colors})

fig = go.Figure(data=[go.Pie(labels=X, values=Y)])
fig.update_traces(marker=dict(colors=colors))
fig.show()
#px.pie(counts, values='Население', names='Регион', color='color')